# RAGSTA Bot - Powered by RAG
# Author: Vigneswaran

### Project Overview:
 RAGSTA Bot is a Retrieval-Augmented Generation (RAG) based chatbot that provides accurate responses to user queries about a given dataset or document. The bot uses a combination of pre-existing knowledge and real-time information retrieval to generate contextually relevant and informative answers.


### Key Features:
* Utilizes Cohere for natural language processing and embedding generation
* Implements Pinecone for efficient vector storage and similarity search
* Incorporates a custom dataset of Apple-related questions and answers

### Table of Contents:
1. Importing Python Libraries
2. Data Extraction and Preparation
3. Embedding Generation
4. Vector Database Setup
5. Query Processing and Response Generation
6. Example Queries

### 1. Importing Python Libraries
- Install required packages
- Import necessary modules for the project
- Initialize Cohere & Pinecone with their API Keys

In [1]:
# Install required packages
%pip install -U cohere pinecone numpy pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import cohere
import pandas as pd
import numpy as np
from pinecone import Pinecone, ServerlessSpec, PineconeApiException

In [3]:
# Initialize Cohere client
co = cohere.Client("COHERE_API_KEY")

In [4]:
# Initialize Pinecone
pc = Pinecone(api_key='PINECONE_API_KEY')

### 2. Data Extraction and Preparation
- Load the dataset from a CSV file
- Prepare the data for embedding generation

In [5]:
# Load the dataset
df = pd.read_csv("C:/Users/Vicky/Documents/RAG Bot/notebook/dataset.csv",header=0)
df.shape

(1005, 2)

In [6]:
# Display the first few questions in the dataset
print(df.iloc[:, 0].head())

0      How do I take a screenshot on an iPhone?
1    How do I change my wallpaper on an iPhone?
2      How do I make a phone call on an iPhone?
3    How do I send a text message on an iPhone?
4               How do I use Siri on an iPhone?
Name: Question, dtype: object


In [7]:
# Combine question and answer columns
df['combined'] = df['Question'].astype(str) + " " + df['Answer'].astype(str)
texts = df['combined'].tolist()

### 3. Embedding Generation
- Generate embeddings for the combined text using Cohere
- Verify the shape of the generated embeddings

In [8]:
# Generate embeddings
embeds = co.embed(
    texts=texts,
    model='embed-english-v3.0',
    input_type='search_document',
    truncate='END'
).embeddings

# Print the shape of the embeddings
embed_shape = np.array(embeds).shape
print(f"Embedding shape: {embed_shape}")

Embedding shape: (1005, 1024)


### 4. Vector Database Setup
- Create a new index or connect to an existing one
- Upsert vectors into the index

In [9]:
# Set up index parameters
index_name = "quickstart"
dimension = 1024
metric = "cosine"


try:
    # Check if the index exists by connecting
    index = pc.Index(index_name)
    print(f"Connected to the existing index: {index_name}")
except PineconeApiException as e:
    # If index doesn't exist, create a new one
    print(f"Index {index_name} not found, creating a new index.")
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    index = pc.Index(index_name)
    print(f"Created and connected to the new index: {index_name}")

Connected to the existing index: quickstart


In [10]:
# Prepare data for upserting
batch_size = 128
ids = [str(i) for i in range(len(embeds))]
df['Question'] = df['Question'].fillna('')
df['Answer'] = df['Answer'].fillna('')
meta = [{'question': q, 'answer': a} for q, a in zip(df['Question'], df['Answer'])]
to_upsert = list(zip(ids, embeds, meta))

# Upsert vectors in batches
for i in range(0, len(embeds), batch_size):
    i_end = min(i + batch_size, len(embeds))
    index.upsert(vectors=to_upsert[i:i_end])

# Print index statistics
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1005}},
 'total_vector_count': 1005}


### 5. Query Processing and Response Generation
- Process user queries
- Retrieve relevant context from the vector database
- Generate responses using Cohere's language model

In [11]:
def process_query(query):
    # Generate query embedding
    xq = co.embed(
        texts=[query],
        model='embed-english-v3.0',
        input_type='search_query',
        truncate='END'
    ).embeddings

    # Query the index
    res = index.query(vector=xq, top_k=5, include_metadata=True)

    # Retrieve relevant context
    context = ""
    for match in res['matches']:
        context += f"Question: {match['metadata']['question']}\n"
        context += f"Answer: {match['metadata']['answer']}\n\n"

    # Combine query and context for the language model
    combined_message = f"User Query: {query}\n\nRelevant Context:\n{context}"

    # Generate response using Cohere's language model
    response = co.chat(message=combined_message, model="command-r-plus")

    return response.text


### 6. Example Queries

In [12]:
# Test the RAGSTA Bot with sample queries
query="How to download apps"
print(f"Query: {query}")
response = process_query(query)
print(f"Response: {response}\n")

Query: How to download apps
Response: Downloading apps on your iPhone is simple and can be done through the App Store. Here's a step-by-step guide on how to do it: 

**Step 1: Open the App Store**

Locate the App Store icon on your iPhone and tap to open it. 

**Step 2: Search for an App**

- Use the search bar at the top of the screen to look for a specific app by name. 
- Alternatively, browse through the various categories, such as Games, Social, or Utilities, to explore different types of apps. You can also check out the featured apps on the Today tab or see what's popular in the charts. 

**Step 3: Choose an App**

Once you've found an app you like, tap on it to open the app's product page. Here, you can read reviews, check the developer's description, and view screenshots or a preview of the app. 

**Step 4: Download the App**

- If the app is free: Tap "Get" and then "Install." The app will start downloading immediately. 
- If the app has a price: Tap the price, and a pop-up wil

In [13]:
query="How to take a photo"
print(f"Query: {query}")
response = process_query(query)
print(f"Response: {response}\n")

Query: How to take a photo
Response: Here is a step-by-step guide on how to take a photo using your device's built-in Camera app: 

1. Open the Camera App: Find and tap on the camera icon on your device to open the Camera app. 

2. Frame Your Shot: Hold your device steady and point it towards the subject or scene you want to capture. Move closer or farther away, and adjust your angle to get the composition you desire. 

3. Focus and Exposure: Tap on the screen where you want the camera to focus. The camera will automatically adjust the exposure and focus settings for that area. You can also hold and drag the sun icon next to the focus box to manually adjust the exposure if needed. 

4. Choose Camera Mode (Optional): Depending on the lighting conditions and the type of photo you want to take, you can choose from various camera modes. For example, Portrait mode is excellent for close-up shots with a blurred background, while Panorama mode captures a wide-angle view. 

5. Tap the Shutter 

In [14]:
query="How to use Siri"
print(f"Query: {query}")
response = process_query(query)
print(f"Response: {response}\n")

Query: How to use Siri
Response: Here is a comprehensive guide on how to use Siri on your iPhone, as well as some tips for using Siri on your Apple Watch: 

**Using Siri on iPhone:**

Activating Siri:
- For older iPhones (with a Home button): Press and hold the Home button until you hear a chime and see the Siri interface. 
  - You can also enable the "Hey Siri" feature by going to Settings > Siri & Search, and then saying "Hey Siri" to activate Siri without pressing any buttons.
- For newer iPhones (without a Home button): Press and hold the Side button until you hear a chime and the Siri interface appears. 
  - Similarly, you can enable "Hey Siri" for a hands-free experience.

Giving Commands:
- Once Siri is activated, you can give various voice commands. Here are some examples: 
  - Making Calls: Say something like, "Call Mom," and Siri will initiate a call to your mother's number. 
  - Sending Texts: You can send a text by saying, "Send a message to Dad saying, 'Running late. Be th

In [15]:
# Test the RAGSTA Bot with multiple queries

example_queries = [
    "How to save photos from a browser?",
    "What's the best way to extend battery life on an iPhone?",
    "How can I set up Apple Pay on my device?"
]

for query in example_queries:
    print(f"Query: {query}")
    response = process_query(query)
    print(f"Response: {response}\n")

Query: How to save photos from a browser?
Response: To save photos from a web browser, simply follow these steps:

 For iPhone/iPad (using Safari): 

- Open the Safari browser and navigate to the web page containing the photo you wish to save.
- Tap and hold the image until a menu appears.
- Select "Save Image" from the menu to download the photo to your camera roll. 

For PC (using Windows and Google Chrome):

- Open Google Chrome and navigate to the web page with the desired photo.
- Right-click on the image and select "Save Image As" from the context menu.
- Choose the location on your computer where you want to save the photo and click "Save." 

For Mac (using Safari): 

- Launch Safari and navigate to the webpage with the photo you want to save.
- Control-click or right-click on the image, then choose "Save Image to Downloads" or "Save Image As..." from the menu that appears. 
- Specify the save location if needed and click "Save." 

With these steps, you can easily save any photo